## Critical path showcase



This notebook provides examples of a simulation that takes a number of barges and a total amount to be moved. We show the critical path (as a pd.DataFrame and a gantt plot) for a number of different simulation (data) and methods.

For this example we work with the following methods to determine critical path:
* __DependenciesFromRecordedActivities__
* __DependenciesFromSimpy__

The latter method is recommended as the dependencies found with `from simpy` correspond wth the events as scheduled within the simulation, whereas `from recorded activities` assumes dependencies from recorded data (dependencies assumed in case of identical timestamps and simulation objects).

For this example we work with the following simulations to determine critical path:
* two barges move an amount from site 0 to site 1. When the container on the site 1 reaches a certain level, another (third) vessel may move the amount from site 1 to site 2. The barges do not arrive nor relieve one another at sites at identical times and (hence) the dependencies are identical for both methods.
* four barges move an amount from site 0 to site 1. When the container on the site 1 reaches a certain level, another (fifth) vessel moves the entire amount from site 1 to site 2. The barges do arrive or relieve each other at sites at identical times - which causes more dependencies with method `from recorded activities` than with method `from simpy`.

Note that in the two barges simulation the barges have an unqiue start event (delay) and velocity - for the four barges simulation this is not the case.

#### 0. Import libraries

In [1]:
import simpy
import shapely

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

from openclsim.critical_path.dependencies_from_simpy_step import DependenciesFromSimpy, CriticalPathEnvironment
from openclsim.critical_path.dependencies_from_recorded_activities import DependenciesFromRecordedActivities


#### 1. Use a function definition to get some simulation data

In this notebook we can play around with the nr of barges and total amount to be moved. After calling this function with desired input parameters we can investigate the data of the simulations and inspect the gantt chart.

In [2]:
def demo_data(nr_barges, total_amount, env=None):
    """
    Run a simulation where <nr_barges> barges need to shift an amount of <total_amount>
    from site 1 (nr resources=1) to site 2 (nr resources=4) whereafter another vessel can come into action.

    Parameters
    ----------
    nr_barges : int
        Number of barges in the simulation.
    total_amount : int
        Total amount to be transported in the simulation.
    env : simpy.Environment or class that inherits from simpy.Environment
        Optional. If None, default to simpy Environment
    """
    Site = type(
        "Site",
        (
            core.Identifiable,
            core.Log,
            core.Locatable,
            core.HasContainer,
            core.HasResource,
        ),
        {},
    )
    TransportProcessingResource = type(
        "TransportProcessingResource",
        (
            core.ContainerDependentMovable,
            core.Processor,
            core.HasResource,
            core.Identifiable,
            core.Log,
        ),
        {},
    )

    simulation_start = 0
    if env is None:
        my_env = simpy.Environment(initial_time=simulation_start)
    else:
        my_env = env(initial_time=simulation_start)

    registry = {}

    location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

    data_from_site = {
        "env": my_env,
        "name": "from_site",
        "geometry": location_from_site,
        "capacity": 4 * total_amount,
        "level": 4 * total_amount,
        "nr_resources": 1,
    }
    from_site = Site(**data_from_site)

    location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
    data_to_site = {
        "env": my_env,
        "name": "to_site",
        "geometry": location_to_site,
        "capacity": 3*total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site = Site(**data_to_site)

    location_to_site2 = shapely.geometry.Point(4.35222222, 52.11428333)
    data_to_site2 = {
        "env": my_env,
        "name": "to_site2",
        "geometry": location_to_site2,
        "capacity": total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site2 = Site(**data_to_site2)

    vessels = {}

    for i in range(nr_barges):
        vessels[f"vessel{i}"] = TransportProcessingResource(
            env=my_env,
            name=f"barge_{i}",
            geometry=location_from_site,
            capacity=10,
            compute_v=lambda x: 10,
        )

    # vessel_last wait till whiletask done
    vessel_last = TransportProcessingResource(
        env=my_env,
        name="vessel_last",
        geometry=location_from_site,
        capacity=10,
        compute_v=lambda x: 10,
    )
    vessels["vessel_last"] = vessel_last

    activities = {}
    for i in range(nr_barges):            
        amount = 5  # handle loading
        duration = 2000  # sailing and unloading
        
        if nr_barges == 2:
            # alter velocity and start event delay based on index barge
            duration_move = duration - 200*i
            start_time = i*100
        else:
            duration_move = duration
            start_time = 0

        requested_resources = {}
        activities[f"activity{i}"] = model.WhileActivity(
            env=my_env,
            name=f"while_sequential_activity_subcycle{i}",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name=f"sequential_activity_subcycle{i}",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessels[f"vessel{i}"]],
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=from_site,
                            duration=duration_move
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=from_site,
                            destination=vessels[f"vessel{i}"],
                            amount=amount,
                            duration=500 * amount,
                            requested_resources=requested_resources,
                            start_event=[
                            {
                                "type": "container",
                                "concept": from_site,
                                "state": "gt",
                                "level": amount
                            }
                            ]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=vessels[f"vessel{i}"],
                            destination=to_site,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": from_site,
                    "state": "lt",
                    "level": 2.8*total_amount
                }
            ],
              start_event={
                "type":"time",
                "start_time":start_time,
            }
        )

    # now add activity for vessel last, once v1 and v2 are done
    requested_resources = {}
    amount = 5
    duration = 2000
    activities[f"activity_last_vessel"] = model.WhileActivity(
            env=my_env,
            name=f"while_last_vessel",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name="sequential_last_vessel",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity vessel_last",
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessel_last]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty: vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=to_site,
                            destination=vessel_last,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site2,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=vessel_last,
                            destination=to_site2,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )],
            start_event=[
                {
                    "type": "container",
                     "concept": to_site,
                     "state": "gt",
                     "level": 30
                }
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": to_site2,
                    "state": "full",
                }
            ],)

    model.register_processes(list(activities.values()))
    my_env.run()

    return {
        "env": my_env,
        "object_list": [from_site, to_site, to_site2] + list(vessels.values()),
        "activity_list": list(activities.values()),
    }



#### 2. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 2 barges_

First we run the simulation, then we initialize the Dependencies instance in order to get the recorded activities (with flag 'is_critical') and show these in a Gantt plot.

In [3]:
# Run the simulation.
simulation_data_out = demo_data(2, 100, env=CriticalPathEnvironment)

In [4]:
# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,e69c97bf-685f-4f3d-bcb5-47aea9f342e5,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,f4dc5dc1-3a0b-4623-9a0f-d0b4e938b639,False
9,a4b2f716-0ae8-4e05-899d-d847fe2b97fd,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,bf11a822-e400-44d8-9932-893a9638c4d7,True
15,3d2972aa-aad7-4a0b-8a35-5bc24481773e,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,6cffdd9f-bef5-41a4-b227-324a5d021920,False
18,8f6d0a03-1081-4134-bbb2-b6e59e524fef,sailing empty:barge_1,barge_1,1970-01-01 00:35:00,1970-01-01 01:05:00,0 days 00:30:00,ACTIVE,7218bf9b-6303-45bb-a3a6-203ca1c8b92b,True
21,aa852ce6-1b16-4604-92a3-b94d942e05a9,loading:barge_1,barge_1,1970-01-01 01:05:00,1970-01-01 01:46:40,0 days 00:41:40,ACTIVE,a792b2c0-f89f-41f7-8eef-eb69cacdbfcc,True


In [5]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()

#### 3. Get critical path with recorded activities

* _method: DependenciesFromRecordedActivities_
* _simulation: simulation with 2 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [6]:
# Run the simulation.
simulation_data_out = demo_data(2, 100)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_activities = DependenciesFromRecordedActivities(**simulation_data_out)
critical_df = my_cp_dependencies_from_activities.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,7dcf020f-e1fc-4247-9bb4-407d81b3e929,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,c7a46a13-1b05-4da2-9e93-55f8b63ed447,False
9,5df0c542-c4a2-4e0b-a803-8fac2a7197da,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,1bdb7844-e19d-4859-b9a3-ec620e654585,False
15,34298b8f-0f0b-4d76-81cf-524b11c27a95,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,dec89b40-3253-4b26-a3bc-c244d02ddb90,False
18,348167c2-d150-4015-8d8f-066d621eb46d,sailing empty:barge_1,barge_1,1970-01-01 00:35:00,1970-01-01 01:05:00,0 days 00:30:00,ACTIVE,31abdcd0-fa10-4d0e-968d-e251cc8074e7,False
21,5694cea9-1f8f-4110-a32c-95d90b06e5ec,loading:barge_1,barge_1,1970-01-01 01:05:00,1970-01-01 01:46:40,0 days 00:41:40,ACTIVE,7f60ba2b-303c-4cad-a575-0cd5b964cc8c,False


In [7]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()

#### 4. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 4 barges_

First we run the simulation, then we initialize the Dependencies instance in order to get the recorded activities (with flag 'is_critical') and show these in a Gantt plot.

In [8]:
# Run the simulation.
simulation_data_out = demo_data(4, 100, env=CriticalPathEnvironment)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,fda44a86-962b-4009-8005-d67cf0dd831f,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,e73c31bc-14e6-4b42-984c-88ec612d297a,True
3,c784587e-639b-4ae6-8e06-ca819f785a53,basic activity:barge_1,barge_1,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,78a3b9a9-c23d-4325-9fca-826b0ae35d03,False
5,5f770e9a-eb28-47e9-8128-8d145d8340b3,basic activity:barge_2,barge_2,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,8f2048f3-dc59-43de-9ed6-9e1eb2361e1f,False
7,23213ef6-87d1-4528-9cef-d7eec44af75e,basic activity:barge_3,barge_3,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,8aba5946-35ce-45e1-8b6e-023d8fccf75e,False
26,7de100d0-989d-4199-8896-0f58e04f5c75,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,c3b801c7-3e10-4a62-a27a-7b8dfc92057f,True


In [9]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()

#### 3. Get critical path with recorded activities

* _method: DependenciesFromRecordedActivities_
* _simulation: simulation with 4 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [10]:
# Run the simulation.
simulation_data_out = demo_data(4, 100)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromRecordedActivities(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,77728539-b637-4a53-becd-133412f2d1a9,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,3d83add2-b1ec-473a-840a-a2d1666faebc,True
3,70d2178e-ae6c-44da-a5d2-043b18a2e30c,basic activity:barge_1,barge_1,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,66be7634-f3a8-4058-a63c-ad2b6c60f5b6,False
5,71f0192b-0b4e-4e27-b53a-bd4d3278b1cf,basic activity:barge_2,barge_2,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,c696ee14-8960-4e9a-a35e-f53f2101ed82,False
7,2c99d08d-6681-4af5-82d0-9ab966630d8b,basic activity:barge_3,barge_3,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,2f14375b-ab84-4682-a1ed-b85628e602a9,False
26,8dcbb26f-2c84-40e3-99e3-cf0407570271,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,929cfa8d-cfb3-4e5d-8079-402c7fe2b212,True


In [11]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()